In [21]:
import os, re
import pandas as pd

In [58]:
exp_lib

,#,Sample name,drug_name,rep,gDNA conc,U6 Bc primer,Read 1 beginning sequence,Index 1 primer,Index 1 (i7) sequence,Index 2 primer,Index 2 (i5) sequence,Index 2 (i5) revcomp,Desired reads,exptype,r1_file,r2_file
idx_code,,,,,,,,,,,,,,,,
ATTACTCG:AGGATAGG,19,mESC_1bpSalib_-ku60019_rep1,1bpSalib_-ku60019,1,NaN,NaN,NaN,i701,ATTACTCG,i503,AGGATAGG,CCTATCCT,"2,500,000",-ku60019,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...
CGCTCATT:AGGATAGG,21,mESC_1bpSalib_+ku60019_rep1,1bpSalib_-ku60019,1,NaN,NaN,NaN,i703,CGCTCATT,i503,AGGATAGG,CCTATCCT,"2,500,000",+ku60019,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...
GAGATTCC:AGGATAGG,22,mESC_1bpSalib_+ku60019_rep2,1bpSalib_-ku60019,2,NaN,NaN,NaN,i704,GAGATTCC,i503,AGGATAGG,CCTATCCT,"2,500,000",+ku60019,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...
TCCGGAGA:AGGATAGG,20,mESC_1bpSalib_-ku60019_rep2,1bpSalib_-ku60019,2,NaN,NaN,NaN,i702,TCCGGAGA,i503,AGGATAGG,CCTATCCT,"2,500,000",-ku60019,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...


In [63]:
reads_dir = "/data/cgs/bh0085/shortreads/1219_SEQUENCING_READS/"
dm_reads_dir = "/data/cgs/bh0085/shortreads/1219_SEQUENCING_READS/READS_DEMULTIPLEXED_2/"

if not os.path.isdir(dm_reads_dir):
    os.makedirs(dm_reads_dir)
    
exp_lib = pd.read_csv(os.path.join("../libs/1bp_SaCasgRNA_redlibrary_SequenceSamples.csv"))
exp_idx_code = exp_lib.apply(lambda x:f"{x['Index 1 (i7) sequence']}:{x['Index 2 (i5) sequence']}",axis=1)
exp_lib["idx_code"] = exp_idx_code   
exp_lib = exp_lib.set_index("idx_code").sort_index()
exp_lib["exptype"] =exp_lib.apply(lambda x: "+ku60019" if "+" in x["Sample name"] else "-ku60019" ,axis=1)

exp_lib["r1_file"] = exp_lib.apply(lambda x:os.path.join(dm_reads_dir,f"{x.exptype}_{x.rep}_R1.fastq"),axis=1)
exp_lib["r2_file"] = exp_lib.apply(lambda x:os.path.join(dm_reads_dir,f"{x.exptype}_{x.rep}_R2.fastq"),axis=1)


# for elt in exp_lib.r1_file:
#     if os.path.isfile(elt):os.remove(elt)
# for elt in exp_lib.r2_file:
#     if os.path.isfile(elt):os.remove(elt)
    
for ln in [1,]:
    
    
    fn1 = f"Undetermined_S0_L00{ln}_R1_001.fastq"
    fn2 = f"Undetermined_S0_L00{ln}_R2_001.fastq"

    f1s = exp_lib.r1_file.apply(lambda x:open(x,"a"))
    f2s = exp_lib.r2_file.apply(lambda x:open(x,"a"))
    
    with open(os.path.join(reads_dir,fn1)) as f1:
        with open(os.path.join(reads_dir,fn2)) as f2:
            cnt = -1
            while(1):
                
                cnt+=1
                try:
                    l1 = next(f1)
                    l2 = next(f2)
                except StopIteration:
                    break
                    
                if cnt %1e6 ==0:
                    print(cnt / 150e6)
                    
                if cnt %4 == 0:
                    h1= l1
                    h2= l2
                elif cnt %4==1:
                    r1 = l1
                    r2 = l2
                elif cnt %4==2:
                    q1 = l1
                    q2 = l2
                elif cnt %4==3:
                    match = re.compile("([ATCGN]+)\+([ATCGN]+)").search(h1)
                    i7,i5 = match.groups()
                    code = f"{i7}:{i5}"
                    
                    if(exp_lib.index.contains(code)):
                        exp = exp_lib.loc[code]
                        f1s.loc[code].write("".join([h1,r1,q1,l1])+"")
                        f2s.loc[code].write("".join([h2,r2,q2,l2])+"")
                        

                        


                    
                    


In [57]:
exp_lib.index.contains("ATTACTCG:AGGATAGG")

True

In [17]:
exp_lib

,#,Sample name,drug_name,rep,gDNA conc,U6 Bc primer,Read 1 beginning sequence,Index 1 primer,Index 1 (i7) sequence,Index 2 primer,Index 2 (i5) sequence,Index 2 (i5) revcomp,Desired reads
0,19,mESC_1bpSalib_-ku60019_rep1,1bpSalib_-ku60019,1,NaN,NaN,NaN,i701,ATTACTCG,i503,AGGATAGG,CCTATCCT,"2,500,000"
1,20,mESC_1bpSalib_-ku60019_rep2,1bpSalib_-ku60019,2,NaN,NaN,NaN,i702,TCCGGAGA,i503,AGGATAGG,CCTATCCT,"2,500,000"
2,21,mESC_1bpSalib_+ku60019_rep1,1bpSalib_-ku60019,1,NaN,NaN,NaN,i703,CGCTCATT,i503,AGGATAGG,CCTATCCT,"2,500,000"
3,22,mESC_1bpSalib_+ku60019_rep2,1bpSalib_-ku60019,2,NaN,NaN,NaN,i704,GAGATTCC,i503,AGGATAGG,CCTATCCT,"2,500,000"


In [59]:
exp_lib

,#,Sample name,drug_name,rep,gDNA conc,U6 Bc primer,Read 1 beginning sequence,Index 1 primer,Index 1 (i7) sequence,Index 2 primer,Index 2 (i5) sequence,Index 2 (i5) revcomp,Desired reads,exptype,r1_file,r2_file
idx_code,,,,,,,,,,,,,,,,
ATTACTCG:AGGATAGG,19,mESC_1bpSalib_-ku60019_rep1,1bpSalib_-ku60019,1,NaN,NaN,NaN,i701,ATTACTCG,i503,AGGATAGG,CCTATCCT,"2,500,000",-ku60019,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...
CGCTCATT:AGGATAGG,21,mESC_1bpSalib_+ku60019_rep1,1bpSalib_-ku60019,1,NaN,NaN,NaN,i703,CGCTCATT,i503,AGGATAGG,CCTATCCT,"2,500,000",+ku60019,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...
GAGATTCC:AGGATAGG,22,mESC_1bpSalib_+ku60019_rep2,1bpSalib_-ku60019,2,NaN,NaN,NaN,i704,GAGATTCC,i503,AGGATAGG,CCTATCCT,"2,500,000",+ku60019,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...
TCCGGAGA:AGGATAGG,20,mESC_1bpSalib_-ku60019_rep2,1bpSalib_-ku60019,2,NaN,NaN,NaN,i702,TCCGGAGA,i503,AGGATAGG,CCTATCCT,"2,500,000",-ku60019,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...


In [61]:
              

SEQUENCING_INFO = pd.DataFrame()

reads_place = dm_reads_dir
reads_files = os.listdir(reads_place)
info = exp_lib

info["s_num"] = info["#"] - 19
info["r1_fastq_path"] = info.r1_file
info["r2_fastq_path"] = info.r2_file

info["nm"] = "all"
info["reads_place"] = reads_place
info["Name"] =  info["Sample name"]
info["Description"] = info["exptype"]

SEQUENCING_INFO = SEQUENCING_INFO.append(info)



In [62]:
SEQUENCING_INFO

,#,Sample name,drug_name,rep,gDNA conc,U6 Bc primer,Read 1 beginning sequence,Index 1 primer,Index 1 (i7) sequence,Index 2 primer,...,exptype,r1_file,r2_file,s_num,r1_fastq_path,r2_fastq_path,nm,reads_place,Name,Description
idx_code,,,,,,,,,,,,,,,,,,,,,
ATTACTCG:AGGATAGG,19,mESC_1bpSalib_-ku60019_rep1,1bpSalib_-ku60019,1,NaN,NaN,NaN,i701,ATTACTCG,i503,...,-ku60019,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...,0,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...,all,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...,mESC_1bpSalib_-ku60019_rep1,-ku60019
CGCTCATT:AGGATAGG,21,mESC_1bpSalib_+ku60019_rep1,1bpSalib_-ku60019,1,NaN,NaN,NaN,i703,CGCTCATT,i503,...,+ku60019,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...,2,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...,all,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...,mESC_1bpSalib_+ku60019_rep1,+ku60019
GAGATTCC:AGGATAGG,22,mESC_1bpSalib_+ku60019_rep2,1bpSalib_-ku60019,2,NaN,NaN,NaN,i704,GAGATTCC,i503,...,+ku60019,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...,3,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...,all,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...,mESC_1bpSalib_+ku60019_rep2,+ku60019
TCCGGAGA:AGGATAGG,20,mESC_1bpSalib_-ku60019_rep2,1bpSalib_-ku60019,2,NaN,NaN,NaN,i702,TCCGGAGA,i503,...,-ku60019,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...,1,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...,all,/data/cgs/bh0085/shortreads/1219_SEQUENCING_RE...,mESC_1bpSalib_-ku60019_rep2,-ku60019


In [12]:
!ls ../libs/

1bp_SaCasgRNA_redlibrary_SequenceSamples.csv


In [8]:
print(fn1)

Undetermined_S0_L001_R1_001.fastq
